In [1]:
#need to import Drive

from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google'

In [ ]:
#This just will show the images

import cv2
from matplotlib import pyplot as plt

img = cv2.imread('/content/drive/MyDrive/chexpert_small/train/patient02330/study1/view1_frontal.jpg',1)
#cv2.imshow()
plt.imshow(img)
plt.axis("off")
plt.show()

In [ ]:
from os import walk
for (dirpath, dirnames, filenames) in walk("../input/"):
    print("Directory path: ", dirpath)
    print("Folder name: ", dirnames)
#     print("File name: ", filenames)

In [ ]:
# DO NOT RUN THIS

"""
import torch
from torchvision import datasets, transforms

transform = transforms.Compose([transforms.Resize(255),
                                 transforms.CenterCrop(224),
                                 transforms.ToTensor()])

dataset = datasets.ImageFolder('/content/drive/MyDrive/chexpert_small/train/', transform=transform)
"""

In [ ]:
# import our race data set and load as a dataframe

import pandas as pd

df_train_race = pd.read_excel('/content/drive/MyDrive/chexpert_small/chexpert_race.xlsx')
df_train_race

In [ ]:
# print the unique labels for race - we need to make some determinations here
print(df_train_race['PRIMARY_RACE'].unique())
print((df_train_race['PRIMARY_RACE'] == 'White, non-Hispanic').sum())

In [ ]:
import sys
import os
from pathlib import Path

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torch import optim

import torchvision.transforms as transforms
import torchvision

from fastprogress import master_bar, progress_bar

from PIL import Image


#I used a lot of code from https://www.kaggle.com/code/hmchuong/chexpert-pytorch-densenet121?scriptVersionId=18314696&cellId=15

class ChestXrayDataset(Dataset):
  #creates a Torch Dataset that we can use to do machine learning on

  def __init__(self,folder_dir,dataframe,image_labels,image_size,normalization):
    #a lot of this function is 
    # folder_dir is the directory path to the data
    # dataframe holds patient info and labels
    # it takes in our image labels

    self.image_paths=[]
    #self.image_labels=[]
    self.image_labels=[]

    #This transforms our image, I think we would also need normalization
    image_transformation = [
        transforms.Resize((image_size,image_size)),
        transforms.ToTensor()
    ]

    #this normalizes using some constants from imagenet
    if normalization:
      image_transformation.append(transforms.Normalize(IMAGENET_MEAN,IMAGENET_STD))
    
    self.image_transformation = transforms.Compose(image_transformation)

    #this will index through all the patients from 000001, so forth, adding images from study1
    for index, row in dataframe.iterrows():
      image_path = os.path.join(folder_dir,Path(row['PATIENT']),Path('study1'),Path('view1_frontal.jpg'))
      self.image_paths.append(image_path)
      #in this case I've hard-coded the image_labels to be if the patient is Black or not
      #NOTICE that I'm append a list to a list here
      if ( row["PRIMARY_RACE"] ==  'White' or row["PRIMARY_RACE"] == 'White, non-Hispanic' or row["PRIMARY_RACE"] == 'White or Caucasian'):
      #if ( row["PRIMARY_RACE"] ==  'Black or African American' or row["PRIMARY_RACE"] == 'Black, non-Hispanic'):
        self.image_labels.append([1])
      else:
        self.image_labels.append([0])
      #self.image_labels.append([int(1*((row["PATIENT"] == 'Black or African American') | (row["PATIENT"] == 'Black, non-Hispanic')))])
    
    #print(self.image_paths)

  def __len__(self):
    #I think this is necessary for other things
    return len(self.image_paths)

  def __getitem__(self,index):
    #This is also just necessary for other parts

    # Read image
    image_path = self.image_paths[index]
    image_data = Image.open(image_path).convert("RGB")
    image_data = self.image_transformation(image_data)
      
    return image_data, torch.FloatTensor(self.image_labels[index])



In [ ]:
IMAGE_SIZE = 224                              # Image size (224x224)
IMAGENET_MEAN = [0.485, 0.456, 0.406]         # Mean of ImageNet dataset (used for normalization)
IMAGENET_STD = [0.229, 0.224, 0.225]          # Std of ImageNet dataset (used for normalization)
BATCH_SIZE = 16                              
LEARNING_RATE = 0.001
LEARNING_RATE_SCHEDULE_FACTOR = 0.1           # Parameter used for reducing learning rate
LEARNING_RATE_SCHEDULE_PATIENCE = 5           # Parameter used for reducing learning rate
MAX_EPOCHS = 2                              # Maximum number of training epochs

In [ ]:
race_is_black = ((df_train_race['PRIMARY_RACE'] == 'Black or African American')  | (df_train_race['PRIMARY_RACE'] =='Black, non-Hispanic'))
race_is_black.sum()

In [ ]:
race_is_white = ((df_train_race['PRIMARY_RACE'] == 'White')  | (df_train_race['PRIMARY_RACE'] =='White, non-Hispanic') | (df_train_race['PRIMARY_RACE'] =='White or Caucasian'))
race_is_white.sum()

In [ ]:
df_train_race_mod = df_train_race.sort_values(by=['PATIENT'])[:2000]
df_train_race_mod

In [ ]:
race_is_black = ((df_train_race_mod['PRIMARY_RACE'] == 'Black or African American')  | (df_train_race_mod['PRIMARY_RACE'] =='Black, non-Hispanic'))
race_is_black.sum()

In [ ]:
df_train_race_mod_test = df_train_race.sort_values(by=['PATIENT'])[2000:2400]
test_race_is_black = ((df_train_race_mod_test['PRIMARY_RACE'] == 'Black or African American')  | (df_train_race_mod_test['PRIMARY_RACE'] =='Black, non-Hispanic'))
test_race_is_black.sum()

In [ ]:
'patient00090' < 'patient00010'
len([1,3,4])

In [ ]:
race_is_black.tolist()[0]

In [ ]:
race_is_white = ((df_train_race_mod['PRIMARY_RACE'] == 'White')  | (df_train_race_mod['PRIMARY_RACE'] =='White, non-Hispanic') | (df_train_race_mod['PRIMARY_RACE'] =='White or Caucasian'))
race_is_white.sum()

In [ ]:
train_dataset = ChestXrayDataset("/content/drive/MyDrive/chexpert_small/train", df_train_race_mod, race_is_white, IMAGE_SIZE, True)
#print(ChestXrayDataset.len(train_dataset))

In [ ]:
import numpy as np
print(train_dataset.image_labels)
print(np.array(train_dataset.image_labels).flatten().sum())

In [ ]:
train_dataloader = DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, num_workers=2, shuffle=True, pin_memory=True)

In [ ]:
#I believe that this functions as a test - if this block returns an error, there is a problem with dataloading
for data, label in train_dataloader:
    print(data.size())
    print(label.size())
    break

In [ ]:
#FORGET THE COLUMNS below, I was trying to implement an algorithm to see if it worked, but it crashes

In [ ]:
#device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device = torch.device('cpu')

In [ ]:
def train(model,  data_loader, optimizer, k_epochs = 1, print_every = 2000):

    begin = time.time()
    # loss function is cross-entropy (multiclass logistic)
    loss_fn = nn.CrossEntropyLoss() 

    # optimizer is Adam, which does fancier stuff with the gradients
    
    for epoch in range(k_epochs): 

        running_loss = 0.0

        i = 0

        #for i, data in enumerate(data_loader, 0):
        for data,label in data_loader:  
            #print(data.size())

            # extract a batch of training data from the data loader
            X = data
            y = torch.flatten(label)
            #y = label
            y = y.to(torch.long)
            X = X.to(device)
            y = y.to(device)

            

            # zero out gradients: we're going to recompute them in a moment
            optimizer.zero_grad()

            # compute the loss (forward pass)
            y_hat = model(X)

            #print("Here!")
            #print(y_hat)
            #print(y)

            loss = loss_fn(y_hat, y)

            #print("All good here!")

            # compute the gradient (backward pass)
            loss.backward()

            # Adam uses the gradient to update the parameters
            optimizer.step()

            # print statistics
            running_loss += loss.item()

            # print the epoch, number of batches processed, and running loss 
            # in regular intervals
            if i % print_every == print_every - 1: 
            #if True:   
                print(f'[epoch: {epoch + 1}, batches: {i + 1:5d}], training loss: {running_loss / print_every:.3f}')
                #print(loss.item())
                running_loss = 0.0

            #print(i)
            
            i += 1
    end = time.time()
    print(f'Finished training in {round(end - begin)}s')

In [ ]:
from scipy.sparse import coo_array
def test(model, data_loader):
    correct = 0
    total = 0
    # torch.no_grad creates an environment in which we do NOT store the 
    # computational graph. We don't need to do this because we don't care about 
    # gradients unless we're training

    i = 0

    with torch.no_grad():
        #for data in data_loader:
        for data, label in data_loader:

            #print(data.size())
            #print(label.size())

            X = data
            y = torch.flatten(label)
            #X, y = data
            X = X.to(device)
            y = y.to(device)
            
            # run all the images through the model
            y_hat = model(X)

            # the class with the largest model output is the prediction
            _, predicted = torch.max(y_hat.data, 1)

            # compute the accuracy
            total += y.size(0)
            correct += (predicted == y).sum().item()

            i += 1

    print(i)
    print(correct)
    print(total)
    print(f'Test accuracy: {100 * correct // total} %')

In [ ]:
from torchvision import models

# instead train only the parameters of the final layer
# can be around 50% faster

model = models.resnet18(weights='IMAGENET1K_V1')
model = model.to(device)

# no gradients for any of the model parameters, so no updates

for param in model.parameters():
  param.requires_grad = False

# Parameters of newly constructed modules have requires_grad=True by default
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2)

model = model.to(device)

# Observe that only parameters of final layer are being optimized as
# opposed to before.
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)

In [ ]:
from torchsummary import summary

#summary(model, (3,224,224))

In [ ]:
import time

df_val_race_mod = df_train_race.sort_values(by=['PATIENT'])[2000:2321]

race_is_black_val = ((df_val_race_mod['PRIMARY_RACE'] == 'Black or African American')  | (df_val_race_mod['PRIMARY_RACE'] =='Black, non-Hispanic'))
#race_is_black.sum()

race_is_white_val = ((df_val_race_mod['PRIMARY_RACE'] == 'White')  | (df_val_race_mod['PRIMARY_RACE'] =='White, non-Hispanic') | (df_val_race_mod['PRIMARY_RACE'] =='White or Caucasian'))
print(race_is_white_val.sum())

#val_dataset = ChestXrayDataset("/content/drive/MyDrive/chexpert_small/train", df_val_race_mod, race_is_black_val, IMAGE_SIZE, True)

val_dataset = ChestXrayDataset("/content/drive/MyDrive/chexpert_small/train", df_val_race_mod, race_is_white_val, IMAGE_SIZE, True)

val_dataloader = DataLoader(dataset=val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2, pin_memory=True)

train(model, train_dataloader, optimizer, k_epochs = 5, print_every = 40)
test(model,val_dataloader)

In [ ]:
df_val_race_mod = df_train_race.sort_values(by=['PATIENT'])[2000:2321]

race_is_black_val = ((df_val_race_mod['PRIMARY_RACE'] == 'Black or African American')  | (df_val_race_mod['PRIMARY_RACE'] =='Black, non-Hispanic'))

print(race_is_black_val.sum())
print(df_val_race_mod.shape)
print(100*(1-race_is_black_val.sum()/df_val_race_mod.shape[0]))

val_dataset = ChestXrayDataset("/content/drive/MyDrive/chexpert_small/train", df_val_race_mod, race_is_black_val, IMAGE_SIZE, True)

print(BATCH_SIZE)

val_dataloader = DataLoader(dataset=val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2, pin_memory=True)

test(model,val_dataloader)

In [ ]:
df_val_race_mod = df_train_race.sort_values(by=['PATIENT'])[2000:2321]
df_val_race_mod

In [ ]:
test(model,val_dataloader)

In [ ]:
print(running_loss)

In [ ]:
import torch.nn as nn
import torch.nn.functional as F #this is for relu

class ConvNet(nn.Module): #inherits from nn.module

  def __init__(self):
    super().__init__() #run init method of parent class

    """
    Let's just define the functions we'll use later for forward
    """

    self.conv1 = nn.Conv2d(3, 100, 5) #3 input channels for rgb, 100 convolutional kernels, all size 5 by 5
    self.conv2 = nn.Conv2d(100, 50, 3)
    self.conv3 = nn.Conv2d(50, 20, 3)

    self.pool = nn.MaxPool2d(2,2) #largest pixel value from each 2 by 2 window
    self.fc1 = nn.Linear(13520,80)
    self.fc2 = nn.Linear(80,40)
    self.fc3 = nn.Linear(40,2) #only two outputs for the binary cat/dog label

  def forward(self,x):
    print('x_shape:',x.shape)
    x = self.pool(F.relu(self.conv1(x))) #do kernel convolution to x, then do relu, then do max pooling
    x = self.pool(F.relu(self.conv2(x)))
    x = self.pool(F.relu(self.conv3(x)))

    print('x_shape:',x.shape)

    #DO NOT FORGET TO FLATTEN BEFORE LINEAR LAYERS

    x = torch.flatten(x,1)

    print('x_shape:',x.shape)

    x = F.relu(self.fc1(x))

    print('x_shape:',x.shape)

    x = F.relu(self.fc2(x))

    print('x_shape:',x.shape)

    x = self.fc3(x) #just return the output, no nonlinear lyaers

    print('x_shape:',x.shape)

    return x

model = ConvNet().to(device)

import torch
torch.cuda.empty_cache()

train(model, train_dataloader, optimizer, k_epochs = 100, print_every = 50)

In [ ]:
model = DenseNet121(num_classes = 2 ).to(device)
model.train()
train(model, train_dataloader, optimizer, k_epochs = 100, print_every = 50)

In [ ]:

class DenseNet121(nn.Module):
    def __init__(self, num_classes, is_trained=True):
        """
        Init model architecture
        
        Parameters
        ----------
        num_classes: int
            number of classes
        is_trained: bool
            whether using pretrained model from ImageNet or not
        """
        super().__init__()
        
        # Load the DenseNet121 from ImageNet
        self.net = torchvision.models.densenet121(pretrained=is_trained)
        
        # Get the input dimension of last layer
        kernel_count = self.net.classifier.in_features
        
        # Replace last layer with new layer that have num_classes nodes, after that apply Sigmoid to the output
        self.net.classifier = nn.Sequential(nn.Linear(kernel_count, num_classes), nn.Sigmoid())
        
    def forward(self, inputs):
        """
        Forward the netword with the inputs
        """
        return self.net(inputs)

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

In [ ]:
model = DenseNet121(num_classes=2).to(device) #binary classification
model

In [ ]:
sum(p.numel() for p in model.parameters() if p.requires_grad)

In [ ]:
# Loss function
loss_criteria = nn.BCELoss()

# Adam optimizer
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE, betas=(0.9, 0.999), eps=1e-8, weight_decay=1e-5)

# Learning rate will be reduced automatically during training
lr_scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor = LEARNING_RATE_SCHEDULE_FACTOR, patience = LEARNING_RATE_SCHEDULE_PATIENCE, mode = 'max', verbose=True)

In [ ]:
def multi_label_auroc(y_gt, y_pred):
    """ Calculate AUROC for each class

    Parameters
    ----------
    y_gt: torch.Tensor
        groundtruth
    y_pred: torch.Tensor
        prediction

    Returns
    -------
    list
        F1 of each class
    """
    auroc = []
    gt_np = y_gt.to("cpu").numpy()
    pred_np = y_pred.to("cpu").numpy()
    assert gt_np.shape == pred_np.shape, "y_gt and y_pred should have the same size"
    for i in range(gt_np.shape[1]):
        auroc.append(roc_auc_score(gt_np[:, i], pred_np[:, i]))
    return auroc

In [ ]:
def epoch_training(epoch, model, train_dataloader, device, loss_criteria, optimizer, mb):
    """
    Epoch training

    Paramteters
    -----------
    epoch: int
      epoch number
    model: torch Module
      model to train
    train_dataloader: Dataset
      data loader for training
    device: str
      "cpu" or "cuda"
    loss_criteria: loss function
      loss function used for training
    optimizer: torch optimizer
      optimizer used for training
    mb: master bar of fastprogress
      progress to log

    Returns
    -------
    float
      training loss
    """
    # Switch model to training mode
    model.train()
    training_loss = 0 # Storing sum of training losses
   
    # For each batch
    for batch, (images, labels) in enumerate(progress_bar(train_dataloader, parent=mb)):
        
        # Move X, Y  to device (GPU)
        images = images.to(device)
        labels = labels.to(device)
        
        # Clear previous gradient
        optimizer.zero_grad()

        # Feed forward the model
        pred = model(images)
        loss = loss_criteria(pred, labels)

        # Back propagation
        loss.backward()

        # Update parameters
        optimizer.step()

        # Update training loss after each batch
        training_loss += loss.item()

        mb.child.comment = f'Training loss {training_loss/(batch+1)}'

    del images, labels, loss
    if torch.cuda.is_available(): torch.cuda.empty_cache()

    # return training loss
    return training_loss/len(train_dataloader)

In [ ]:
def evaluating(epoch, model, val_loader, device, loss_criteria, mb):
    """
    Validate model on validation dataset
    
    Parameters
    ----------
    epoch: int
        epoch number
    model: torch Module
        model used for validation
    val_loader: Dataset
        data loader of validation set
    device: str
        "cuda" or "cpu"
    loss_criteria: loss function
      loss function used for training
    mb: master bar of fastprogress
      progress to log
  
    Returns
    -------
    float
        loss on validation set
    float
        metric score on validation set
    """

    # Switch model to evaluation mode
    model.eval()

    val_loss = 0                                   # Total loss of model on validation set
    out_pred = torch.FloatTensor().to(device)      # Tensor stores prediction values
    out_gt = torch.FloatTensor().to(device)        # Tensor stores groundtruth values

    with torch.no_grad(): # Turn off gradient
        # For each batch
        for step, (images, labels) in enumerate(progress_bar(val_loader, parent=mb)):
            # Move images, labels to device (GPU)
            images = images.to(device)
            labels = labels.to(device)

            # Update groundtruth values
            out_gt = torch.cat((out_gt,  labels), 0)

            # Feed forward the model
            ps = model(images)
            loss = loss_criteria(ps, labels)

            # Update prediction values
            out_pred = torch.cat((out_pred, ps), 0)

            # Update validation loss after each batch
            val_loss += loss
            mb.child.comment = f'Validation loss {val_loss/(step+1)}'

    # Clear memory
    del images, labels, loss
    if torch.cuda.is_available(): torch.cuda.empty_cache()
    # return validation loss, and metric score
    return val_loss/len(val_loader), np.array(multi_label_auroc(out_gt, out_pred)).mean()

In [ ]:
import time

In [ ]:
!pip install git+https://github.com/fastai/fastai --upgrade

!pip install git+https://github.com/fastai/fastprogress --upgrade

In [ ]:
df_train_race_val = df_train_race.sort_values(by=['PATIENT'])[501:700]
race_is_black = ((df_train_race_val['PRIMARY_RACE'] == 'Black or African American')  | (df_train_race_val['PRIMARY_RACE'] =='Black, non-Hispanic'))

val_dataset = ChestXrayDataset("/content/drive/MyDrive/chexpert_small/train", df_train_race_val, race_is_black, IMAGE_SIZE, True)
val_dataloader = DataLoader(dataset=val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2, pin_memory=True)

In [ ]:
from fastprogress.fastprogress import master_bar, progress_bar
from time import sleep

# Best AUROC value during training
best_score = 0
model_path = "densenet.pth"
training_losses = []
validation_losses = []
validation_score = []


# Config progress bar
mb = master_bar(range(MAX_EPOCHS))
mb.names = ['Training loss', 'Validation loss', 'Validation AUROC']
x = []

nonimproved_epoch = 0
start_time = time.time()

# Training each epoch
for epoch in mb:
    #mb.first_bar.comment = f'Best AUROC score: {best_score}'
    x.append(epoch)

    # Training
    train_loss = epoch_training(epoch, model, train_dataloader, device, loss_criteria, optimizer, mb)
    mb.write('Finish training epoch {} with loss {:.4f}'.format(epoch, train_loss))
    training_losses.append(train_loss)

    # Evaluating
    val_loss, new_score = evaluating(epoch, model, val_dataloader, device, loss_criteria, mb)
    mb.write('Finish validation epoch {} with loss {:.4f} and score {:.4f}'.format(epoch, val_loss, new_score))
    validation_losses.append(val_loss)
    validation_score.append(new_score)

    # Update learning rate
    lr_scheduler.step(new_score)

    # Update training chart
    mb.update_graph([[x, training_losses], [x, validation_losses], [x, validation_score]], [0,epoch+1], [0,1])

    # Save model
    if best_score < new_score:
        mb.write(f"Improve AUROC from {best_score} to {new_score}")
        best_score = new_score
        nonimproved_epoch = 0
        torch.save({"model": model.state_dict(), 
                    "optimizer": optimizer.state_dict(), 
                    "best_score": best_score, 
                    "epoch": epoch, 
                    "lr_scheduler": lr_scheduler.state_dict()}, model_path)
    else: 
        nonimproved_epoch += 1
    if nonimproved_epoch > 10:
        break
        print("Early stopping")
    if time.time() - start_time > 3600*8:
        break
        print("Out of time")

In [ ]:
import json

with open("/var/log/colab-jupyter.log", "r") as fo:
  for line in fo:
    print(json.loads(line)['msg'])

In [ ]:
import cv2
from matplotlib import pyplot as plt

img = cv2.imread(train_dataset.image_paths[446],1)
#cv2.imshow()
plt.imshow(img)
plt.axis("off")
plt.show()